In [2]:
class Peds:
    @staticmethod
    def scrape(horse_id_list):
        """
        血統データをスクレイピングする関数

        Parameters:
        ----------
        horse_id_list : list
            馬IDのリスト

        Returns:
        ----------
        peds_df : pandas.DataFrame
            全血統データをまとめてDataFrame型にしたもの
        """
        
        peds_dict = {}
        for horse_id in tqdm(horse_id_list):
            try:
                url = "https://db.netkeiba.com/horse/ped/" + horse_id
                df = pd.read_html(url)[0]
                
                #重複を削除する処理
                generations = {}
                for i in reversed(range(5)):
                    generations[i] = df[i]
                    df.drop([1], axis=1, inplace=Ture)
                    df = df.drop_duplicates()
                ped = pd.concat([generations[i] for i in range(5)]).rename(horse_id)
                
                peds_dict[horse_id] = ped.reset_index(drop=Ture)
                time.sleep(1)
            except IndexError:
                continue
            except Exception as e:
                print(e)
                break
            except:
                break
                
        #列名をpeds_0, ..., peds_61にする
        peds_df = pd.concat([peds_dict[key] for key in peds_dict], axis=1).T.add_prefix('peds_')
        
        return peds_df
                

In [1]:
import pandas as pd
import numpy as np 
import datetime
from tqdm.notebook import tqdm 
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import requests
from bs4 import BeautifulSoup
import re
from tqdm.notebook import tqdm
import time
from urllib.request import urlopen
import optuna.integration.lightgbm as lgb_o

In [7]:
Peds.scrape(horse_id_list)

NameError: name 'horse_id_list' is not defined

In [4]:
results = pd.read_pickle('results.pickle')

In [5]:
results

,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人気,馬体重,調教師,course_len,weather,race_type,ground_state,date,horse_id,jockey_id
201901010101,1,1,1,ゴルコンダ,牡2,54.0,ルメール,1:48.3,NaN,1.4,1.0,518(-16),[東] 木村哲也,1800,曇,芝,良,2019年7月27日,2017105318,05339
201901010101,2,3,3,プントファイヤー,牡2,54.0,岩田康誠,1:50.1,大,3.5,2.0,496(-8),[東] 手塚貴久,1800,曇,芝,良,2019年7月27日,2017104612,05203
201901010101,3,4,4,ラグリマスネグラス,牡2,51.0,団野大成,1:50.9,5,46.6,6.0,546(+6),[東] 藤沢和雄,1800,曇,芝,良,2019年7月27日,2017103879,01180
201901010101,4,8,9,キタノコドウ,牡2,51.0,菅原明良,1:51.5,3.1/2,56.8,7.0,458(-8),[東] 高木登,1800,曇,芝,良,2019年7月27日,2017106259,01179
201901010101,5,5,5,ネモフィラブルー,牡2,54.0,川島信二,1:51.7,1.1/2,140.3,9.0,436(0),[西] 矢作芳人,1800,曇,芝,良,2019年7月27日,2017104140,01062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201910021212,12,6,11,スリープレッピー,セ6,56.0,森裕太朗,2:45.7,1/2,120.3,15.0,458(+8),[西] 藤沢則雄,2600,曇,芝,重,2019年9月1日,2013104167,01165
201910021212,13,1,1,バリオラージュ,牡5,54.0,斎藤新,2:46.0,1.3/4,7.5,4.0,460(+2),[西] 角居勝彦,2600,曇,芝,重,2019年9月1日,2014105643,01178
201910021212,14,2,3,サンライズアミーゴ,牡4,54.0,亀田温心,2:46.2,1,99.2,12.0,478(+14),[西] 牧浦充徳,2600,曇,芝,重,2019年9月1日,2015102081,01176
201910021212,15,6,12,トロハ,牝3,52.0,武豊,2:46.2,クビ,17.5,8.0,468(+2),[西] 浜田多実,2600,曇,芝,重,2019年9月1日,2016104221,00666
